CAPÍTULO 6 
-
Tidy data (dados organizados)

6.1 Introdução  
O conceito de Hadley Wickham o define como um conceito que atende os seguintes critérios:  
 - Cada linha é uma observação (observation)
 - Cada coluna é uma variável (variable)
 - Cada tipo de unidade de observação forma uma tabela

Este capítulo descreve diversas formas de como organizar os dados conforme identificadas pelo artigo

6.2 Colunas contêm valores, e não variáveis

Os dados podem conter colunas que contêm valores em vez de variáveis. Em geral, é um formato conveniente para a coleta e apresentação dos dados

6.2.1 Mantendo uma coluna fixa

In [145]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Ao observar esse conjunto de dados podemos ver que nem todas as colunas são variáveis, os valores relacionados à renda estão espalhados em varias colunas.  
Esse formato é uma ótima opção para tabelas, mas para analise de dados precisamos reformatar esses dados para que religião, renda e contador sejam variáveis

In [146]:
pew = pd.read_csv('../../data/pew.csv')

print(pew.iloc[:, 0:6])

                   religion  <$10k  $10-20k  $20-30k  $30-40k  $40-50k
0                  Agnostic     27       34       60       81       76
1                   Atheist     12       27       37       52       35
2                  Buddhist     27       21       30       34       33
3                  Catholic    418      617      732      670      638
4        Don’t know/refused     15       14       15       11       10
5          Evangelical Prot    575      869     1064      982      881
6                     Hindu      1        9        7        9       11
7   Historically Black Prot    228      244      236      238      197
8         Jehovah's Witness     20       27       24       24       21
9                    Jewish     19       19       25       25       30
10            Mainline Prot    289      495      619      655      651
11                   Mormon     29       40       48       51       56
12                   Muslim      6        7        9       10        9
13    

Essa vizualização é conhecida como dados "largos" (wide). Para transformar dados largos em dados "longos" (long) precisamos efetuar uma operação unpivot/melt/gather em nosso dataframe.  
O Pandas usa a função melt para reformatar o dataframe de maneira organizada.

melt aceita alguns parâmetros: 
 - id_vars: é um contêiner que representa as variáveis que permanecerão inalteradas
 - value_vars: identifica as colunas que a operação melt será execultada. Por padrão ela será execultada em todas as colunas que não foram especificadas por id_vars
 - var_name: é uma string para o nome da nova coluna quando melt é execultado em value_vars (var_name é uma coluna que mostra o nome da variavel)
 - value_name: é uma string para o nome da nova coluna que representa os valores para var_name (value_name é uma coluna com os valores que está contido dentro da variavel)

In [147]:
#Não precisamor passar value_vars pois queremos pivotear todas as colunas, exeto a coluna religion
pew_long = pd.melt(pew, id_vars='religion')

print(pew_long.head(), '\n')
print(pew_long.tail())

             religion variable  value
0            Agnostic    <$10k     27
1             Atheist    <$10k     12
2            Buddhist    <$10k     27
3            Catholic    <$10k    418
4  Don’t know/refused    <$10k     15 

                  religion            variable  value
175               Orthodox  Don't know/refused     73
176        Other Christian  Don't know/refused     18
177           Other Faiths  Don't know/refused     71
178  Other World Religions  Don't know/refused      8
179           Unaffiliated  Don't know/refused    597


Podemos alterar os defaults de modo que as colunas sejeitas à operação de melt/unpivot sejam nomeadas

In [148]:
pew_long = pd.melt(pew,
                   id_vars='religion',
                   var_name='income',
                   value_name='count')

print(pew_long.head(), '\n')
print(pew_long.tail())

             religion income  count
0            Agnostic  <$10k     27
1             Atheist  <$10k     12
2            Buddhist  <$10k     27
3            Catholic  <$10k    418
4  Don’t know/refused  <$10k     15 

                  religion              income  count
175               Orthodox  Don't know/refused     73
176        Other Christian  Don't know/refused     18
177           Other Faiths  Don't know/refused     71
178  Other World Religions  Don't know/refused      8
179           Unaffiliated  Don't know/refused    597


6.2.2 Mantendo várias colunas fixas

Nem todo conjunto de dados terá uma coluna que permanecerá inalterada em quanto você execulta um unpivot no restante das colunas.

In [149]:
billboard = pd.read_csv('../../data/billboard.csv')

print(billboard.iloc[0:5, 0:16])

   year        artist                    track  time date.entered  wk1   wk2  \
0  2000         2 Pac  Baby Don't Cry (Keep...  4:22   2000-02-26   87  82.0   
1  2000       2Ge+her  The Hardest Part Of ...  3:15   2000-09-02   91  87.0   
2  2000  3 Doors Down               Kryptonite  3:53   2000-04-08   81  70.0   
3  2000  3 Doors Down                    Loser  4:24   2000-10-21   76  76.0   
4  2000      504 Boyz            Wobble Wobble  3:35   2000-04-15   57  34.0   

    wk3   wk4   wk5   wk6   wk7   wk8   wk9  wk10  wk11  
0  72.0  77.0  87.0  94.0  99.0   NaN   NaN   NaN   NaN  
1  92.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
2  68.0  67.0  66.0  57.0  54.0  53.0  51.0  51.0  51.0  
3  72.0  69.0  67.0  65.0  55.0  59.0  62.0  61.0  61.0  
4  25.0  17.0  17.0  31.0  36.0  49.0  53.0  57.0  64.0  


Exemplo: se quisesse criar uma plotagem de faceta com as classificações semanais, a variável de faceta teria de ser uma coluna do dataframe

In [150]:
#fizemos um melt e passamos as colunas que deveriam permanecer inalteradas
#depois passamos um nome para a nova variável das semanas e um nome para os valores desta variável
billboard_long = pd.melt(
    billboard,
    id_vars=['year', 'artist',  'track', 'time', 'date.entered'],
    var_name='week',
    value_name='rating'
)

print(billboard_long.head(), '\n')
print(billboard_long.tail())

   year        artist                    track  time date.entered week  rating
0  2000         2 Pac  Baby Don't Cry (Keep...  4:22   2000-02-26  wk1    87.0
1  2000       2Ge+her  The Hardest Part Of ...  3:15   2000-09-02  wk1    91.0
2  2000  3 Doors Down               Kryptonite  3:53   2000-04-08  wk1    81.0
3  2000  3 Doors Down                    Loser  4:24   2000-10-21  wk1    76.0
4  2000      504 Boyz            Wobble Wobble  3:35   2000-04-15  wk1    57.0 

       year            artist                    track  time date.entered  \
24087  2000       Yankee Grey     Another Nine Minutes  3:10   2000-04-29   
24088  2000  Yearwood, Trisha          Real Live Woman  3:55   2000-04-01   
24089  2000   Ying Yang Twins  Whistle While You Tw...  4:19   2000-03-18   
24090  2000     Zombie Nation            Kernkraft 400  3:30   2000-09-02   
24091  2000   matchbox twenty                     Bent  4:12   2000-04-29   

       week  rating  
24087  wk76     NaN  
24088  wk76     N

6.3 Colunas contendo diversas variáveis

As vezes as colunas podem representar diversas variáveis. Esse formato é comunmente visto quando trabalhamos com dados de saúde.

In [151]:
ebola = pd.read_csv('../../data/country_timeseries.csv')
print(ebola.columns)

Index(['Date', 'Day', 'Cases_Guinea', 'Cases_Liberia', 'Cases_SierraLeone',
       'Cases_Nigeria', 'Cases_Senegal', 'Cases_UnitedStates', 'Cases_Spain',
       'Cases_Mali', 'Deaths_Guinea', 'Deaths_Liberia', 'Deaths_SierraLeone',
       'Deaths_Nigeria', 'Deaths_Senegal', 'Deaths_UnitedStates',
       'Deaths_Spain', 'Deaths_Mali'],
      dtype='object')


In [152]:
print(ebola.iloc[:5, [0,1,2,3,10,11]])

         Date  Day  Cases_Guinea  Cases_Liberia  Deaths_Guinea  Deaths_Liberia
0    1/5/2015  289        2776.0            NaN         1786.0             NaN
1    1/4/2015  288        2775.0            NaN         1781.0             NaN
2    1/3/2015  287        2769.0         8166.0         1767.0          3496.0
3    1/2/2015  286           NaN         8157.0            NaN          3496.0
4  12/31/2014  284        2730.0         8115.0         1739.0          3471.0


Aqui uma coluna contêm duas variáveis, o status individual e o país. Exemplo: Cases_Guinea e Deaths_Guinea

In [153]:
ebola_long = pd.melt(ebola, id_vars=['Date', 'Day'])

print(ebola_long)

            Date  Day      variable   value
0       1/5/2015  289  Cases_Guinea  2776.0
1       1/4/2015  288  Cases_Guinea  2775.0
2       1/3/2015  287  Cases_Guinea  2769.0
3       1/2/2015  286  Cases_Guinea     NaN
4     12/31/2014  284  Cases_Guinea  2730.0
...          ...  ...           ...     ...
1947   3/27/2014    5   Deaths_Mali     NaN
1948   3/26/2014    4   Deaths_Mali     NaN
1949   3/25/2014    3   Deaths_Mali     NaN
1950   3/24/2014    2   Deaths_Mali     NaN
1951   3/22/2014    0   Deaths_Mali     NaN

[1952 rows x 4 columns]


6.3.1 Separar e adicionar colunas individualmente (método simples)

Uma coluna pode ser separada com base no underscore (sublinhado), para dividir em duas variáveis. Neste exemplo iremos dívidir uma coluna para o status e uma coluna para o país.  
Assim como uma Series e um DataFrame possuem seus próprios métodos, uma String também possui metodos próprios e um deles é o "split" que separa uma string com um dado delimitador. Esse delimitador por padrão é um espaço, mas podemos alterar ele, neste caso o delimitador é um underscore

In [154]:
#obtém a coluna variable, acessa os métodos de string e separa com base em um delimitador
variable_split = ebola_long.variable.str.split('_')

print(variable_split.iloc[:])

0       [Cases, Guinea]
1       [Cases, Guinea]
2       [Cases, Guinea]
3       [Cases, Guinea]
4       [Cases, Guinea]
             ...       
1947     [Deaths, Mali]
1948     [Deaths, Mali]
1949     [Deaths, Mali]
1950     [Deaths, Mali]
1951     [Deaths, Mali]
Name: variable, Length: 1952, dtype: object


Depois da separação os valores são devolvidos em uma lista

In [155]:
print(type(variable_split))

print(type(variable_split[0]))

<class 'pandas.core.series.Series'>
<class 'list'>


Agora que a coluna foi separa em várias parte, precisamos atribuir essas parte a uma nova coluna.  
Porém porecisamos separar os elementos de indicie 0 e 1, ou seja separar o status do contry

In [156]:
status_values = variable_split.str.get(0)
contry_values = variable_split.str.get(1)

print(status_values, '\n')
print(contry_values)

0        Cases
1        Cases
2        Cases
3        Cases
4        Cases
         ...  
1947    Deaths
1948    Deaths
1949    Deaths
1950    Deaths
1951    Deaths
Name: variable, Length: 1952, dtype: object 

0       Guinea
1       Guinea
2       Guinea
3       Guinea
4       Guinea
         ...  
1947      Mali
1948      Mali
1949      Mali
1950      Mali
1951      Mali
Name: variable, Length: 1952, dtype: object


In [157]:
"""ebola_long['status'] = status_values
ebola_long['contry'] = contry_values
"""
print(ebola_long.head())

         Date  Day      variable   value
0    1/5/2015  289  Cases_Guinea  2776.0
1    1/4/2015  288  Cases_Guinea  2775.0
2    1/3/2015  287  Cases_Guinea  2769.0
3    1/2/2015  286  Cases_Guinea     NaN
4  12/31/2014  284  Cases_Guinea  2730.0


6.3.2 Separar e combinar em um único passo (método simples)

Já que o vetor é devolvido na mesma ordem que nossos dados podemos concatenar o novo vetor aos dados originais ao invés de criar colunas e depois adicionar os dados separadamente

In [158]:
#o parâmetro "expand=True" faz com que retorne um DataFrame, já ele False (o padrão) devolve uma series de listas
variable_split = ebola_long.variable.str.split('_', expand=True)
variable_split.columns = ['status','country']
ebola_parsed = pd.concat([ebola_long, variable_split], axis=1)

print(ebola_parsed)

            Date  Day      variable   value  status country
0       1/5/2015  289  Cases_Guinea  2776.0   Cases  Guinea
1       1/4/2015  288  Cases_Guinea  2775.0   Cases  Guinea
2       1/3/2015  287  Cases_Guinea  2769.0   Cases  Guinea
3       1/2/2015  286  Cases_Guinea     NaN   Cases  Guinea
4     12/31/2014  284  Cases_Guinea  2730.0   Cases  Guinea
...          ...  ...           ...     ...     ...     ...
1947   3/27/2014    5   Deaths_Mali     NaN  Deaths    Mali
1948   3/26/2014    4   Deaths_Mali     NaN  Deaths    Mali
1949   3/25/2014    3   Deaths_Mali     NaN  Deaths    Mali
1950   3/24/2014    2   Deaths_Mali     NaN  Deaths    Mali
1951   3/22/2014    0   Deaths_Mali     NaN  Deaths    Mali

[1952 rows x 6 columns]


6.3.3 Separar e combinar em um único passo (método mais complicado)

Aproveitando do fato que o resultado da separação devolve uma lista com dois elementos em que cada elemnteo é uma nova coluna, podemos combinar a lista de itens separados com a função zip

In [159]:
constants = ['pi', 'e']
values = ['3.14', '2.718']

#temos que chamar list na função zip para exibir o conteúdo do objeto zip;
#em python 3 zip devolve um iterador
print(list(zip(constants, values)))

[('pi', '3.14'), ('e', '2.718')]


Outra forma de vizualizar o que zip faz é tomar cada contêiner passado para si e empilha-los uns sobre os outros (como se fosse uma concatenação por linha)

Podemos usar o ebola_long.variable.str.split(' ') para separar os valores da coluna. Porém  o resultado já é um contêiner, e precisamos  
descompactá-lo para obter o seu conteúdo (cada lista status-país).

Em python o oprador asterisco é usado para desempacotar contêineres. Quando executamos zip nos contêineres desempacotados  
o efeito é o mesmo obtido quando criamos valores de status país separadamente anteriormente.  
Depois, podemos então atribuir os vetores as colunas simultaneamente usando atribuição múltipla

In [160]:
ebola_long['status'], ebola_long['country'] = zip(*ebola_long.variable.str.split('_'))

print(ebola_long)

            Date  Day      variable   value  status country
0       1/5/2015  289  Cases_Guinea  2776.0   Cases  Guinea
1       1/4/2015  288  Cases_Guinea  2775.0   Cases  Guinea
2       1/3/2015  287  Cases_Guinea  2769.0   Cases  Guinea
3       1/2/2015  286  Cases_Guinea     NaN   Cases  Guinea
4     12/31/2014  284  Cases_Guinea  2730.0   Cases  Guinea
...          ...  ...           ...     ...     ...     ...
1947   3/27/2014    5   Deaths_Mali     NaN  Deaths    Mali
1948   3/26/2014    4   Deaths_Mali     NaN  Deaths    Mali
1949   3/25/2014    3   Deaths_Mali     NaN  Deaths    Mali
1950   3/24/2014    2   Deaths_Mali     NaN  Deaths    Mali
1951   3/22/2014    0   Deaths_Mali     NaN  Deaths    Mali

[1952 rows x 6 columns]


6.4 Variáveis tanto em linhas quanto em colunas

Esta subseção ira mostrar como organizar os dados caso hajá duas variáveis em uma única e coluna, e varias colunas para uma única variável

In [161]:
weather = pd.read_csv('../../data/weather.csv')

print(weather.iloc[:5, :11])

        id  year  month element  d1    d2    d3  d4    d5  d6  d7
0  MX17004  2010      1    tmax NaN   NaN   NaN NaN   NaN NaN NaN
1  MX17004  2010      1    tmin NaN   NaN   NaN NaN   NaN NaN NaN
2  MX17004  2010      2    tmax NaN  27.3  24.1 NaN   NaN NaN NaN
3  MX17004  2010      2    tmin NaN  14.4  14.4 NaN   NaN NaN NaN
4  MX17004  2010      3    tmax NaN   NaN   NaN NaN  32.1 NaN NaN


Neste caso a coluna element possuí duas variáveis que precisam ser submetidas a cast/pivot para separar as variáveis de tmax e tmin (temperatura máxima e mínima).  
Já as variáveis de dias precisam ser submetidas a um melt, para transformar em uma única coluna com os dias

Primeiro vamos execultar a operação de melt nos dias

In [162]:
weather_melt = pd.melt(weather,
                       id_vars=['id','year','month','element'],
                       var_name='day',
                       value_name='temp')

print(weather_melt)

          id  year  month element  day  temp
0    MX17004  2010      1    tmax   d1   NaN
1    MX17004  2010      1    tmin   d1   NaN
2    MX17004  2010      2    tmax   d1   NaN
3    MX17004  2010      2    tmin   d1   NaN
4    MX17004  2010      3    tmax   d1   NaN
..       ...   ...    ...     ...  ...   ...
677  MX17004  2010     10    tmin  d31   NaN
678  MX17004  2010     11    tmax  d31   NaN
679  MX17004  2010     11    tmin  d31   NaN
680  MX17004  2010     12    tmax  d31   NaN
681  MX17004  2010     12    tmin  d31   NaN

[682 rows x 6 columns]


Insight foda que tive agora
-

    -melt = transforma as colunas em valores dentro de uma só coluna
    
    -pivot = transforma valores dentro de uma coluna em uma coluna única

Agora vamos pivotear as variáveis armazenadas na coluna element

In [163]:
weather_tidy = weather_melt.pivot_table(
    index=['id', 'year', 'month', 'day'],
    columns='element',
    values='temp'
)
weather_tidy_flat = weather_tidy.reset_index()

print(weather_tidy_flat)

element       id  year  month  day  tmax  tmin
0        MX17004  2010      1  d30  27.8  14.5
1        MX17004  2010      2  d11  29.7  13.4
2        MX17004  2010      2   d2  27.3  14.4
3        MX17004  2010      2  d23  29.9  10.7
4        MX17004  2010      2   d3  24.1  14.4
5        MX17004  2010      3  d10  34.5  16.8
6        MX17004  2010      3  d16  31.1  17.6
7        MX17004  2010      3   d5  32.1  14.2
8        MX17004  2010      4  d27  36.3  16.7
9        MX17004  2010      5  d27  33.2  18.2
10       MX17004  2010      6  d17  28.0  17.5
11       MX17004  2010      6  d29  30.1  18.0
12       MX17004  2010      7   d3  28.6  17.5
13       MX17004  2010      7  d14  29.9  16.5
14       MX17004  2010      8  d23  26.4  15.0
15       MX17004  2010      8   d5  29.6  15.8
16       MX17004  2010      8  d29  28.0  15.3
17       MX17004  2010      8  d13  29.8  16.5
18       MX17004  2010      8  d25  29.7  15.6
19       MX17004  2010      8  d31  25.4  15.4
20       MX17

Do mesmo modo também podemos aplicar esses métodos sem o dataframe intermediário:

In [164]:
weather_tidy = weather_melt.pivot_table(
    index=['id','year','month','day'],
    columns='element',
    values='temp')\
        .reset_index()

print(weather_tidy)

element       id  year  month  day  tmax  tmin
0        MX17004  2010      1  d30  27.8  14.5
1        MX17004  2010      2  d11  29.7  13.4
2        MX17004  2010      2   d2  27.3  14.4
3        MX17004  2010      2  d23  29.9  10.7
4        MX17004  2010      2   d3  24.1  14.4
5        MX17004  2010      3  d10  34.5  16.8
6        MX17004  2010      3  d16  31.1  17.6
7        MX17004  2010      3   d5  32.1  14.2
8        MX17004  2010      4  d27  36.3  16.7
9        MX17004  2010      5  d27  33.2  18.2
10       MX17004  2010      6  d17  28.0  17.5
11       MX17004  2010      6  d29  30.1  18.0
12       MX17004  2010      7   d3  28.6  17.5
13       MX17004  2010      7  d14  29.9  16.5
14       MX17004  2010      8  d23  26.4  15.0
15       MX17004  2010      8   d5  29.6  15.8
16       MX17004  2010      8  d29  28.0  15.3
17       MX17004  2010      8  d13  29.8  16.5
18       MX17004  2010      8  d25  29.7  15.6
19       MX17004  2010      8  d31  25.4  15.4
20       MX17

6.5 Várias unidades de observação em uma tabela (normalização)

Um dos modos mais simples de saber se várias unidades de observação estão representadas em uma tabela, é observar se alguma cécula ou valor está sendo repetida nas linhas 

In [165]:
print(billboard_long.head())

   year        artist                    track  time date.entered week  rating
0  2000         2 Pac  Baby Don't Cry (Keep...  4:22   2000-02-26  wk1    87.0
1  2000       2Ge+her  The Hardest Part Of ...  3:15   2000-09-02  wk1    91.0
2  2000  3 Doors Down               Kryptonite  3:53   2000-04-08  wk1    81.0
3  2000  3 Doors Down                    Loser  4:24   2000-10-21  wk1    76.0
4  2000      504 Boyz            Wobble Wobble  3:35   2000-04-15  wk1    57.0


Suponha que tenhamos criado um subconjunto dos dados com base em uma faixa musical particular

In [166]:
print(billboard_long[billboard_long.track == 'Loser'].head())

      year        artist  track  time date.entered week  rating
3     2000  3 Doors Down  Loser  4:24   2000-10-21  wk1    76.0
320   2000  3 Doors Down  Loser  4:24   2000-10-21  wk2    76.0
637   2000  3 Doors Down  Loser  4:24   2000-10-21  wk3    72.0
954   2000  3 Doors Down  Loser  4:24   2000-10-21  wk4    69.0
1271  2000  3 Doors Down  Loser  4:24   2000-10-21  wk5    67.0


Podemos ver que essa tabela armazena dois tipos de dados, a informação da faixa musical e a classificação semanal. Mas seria melhor armazenar as informações das colunas year, artitst, track, time e data.entered em outro dataframe já que repetir os mesmos valores de modo contínuo eleva os riscos de haver dados inconsistentes.  
Neste caso o que podemos fazer é colocar esses conjuntos únicos de valores em um dataframe novo e dar a eles um ID único

In [167]:
billboard_songs = billboard_long[['year','artist','track','time']]
print(billboard_songs.shape)

(24092, 4)


In [168]:
billboard_songs = billboard_songs.drop_duplicates()

print(billboard_songs.shape)

(317, 4)


Agora podemos atribuir um valor único a cada linha de dados

In [169]:
billboard_songs['id'] = range(len(billboard_songs))

print(billboard_songs.head(n=10))

   year          artist                    track  time  id
0  2000           2 Pac  Baby Don't Cry (Keep...  4:22   0
1  2000         2Ge+her  The Hardest Part Of ...  3:15   1
2  2000    3 Doors Down               Kryptonite  3:53   2
3  2000    3 Doors Down                    Loser  4:24   3
4  2000        504 Boyz            Wobble Wobble  3:35   4
5  2000            98^0  Give Me Just One Nig...  3:24   5
6  2000         A*Teens            Dancing Queen  3:44   6
7  2000         Aaliyah            I Don't Wanna  4:15   7
8  2000         Aaliyah                Try Again  4:03   8
9  2000  Adams, Yolanda            Open My Heart  5:30   9


Agora que temos um dataframe separado para as músicas podemos usar a coluna id recém criada para fazer uma correspondência entre uma musica e sua classificação semanal

In [170]:
# Combina o dataframe de músicas com o conjunto de dados original
billboard_ratings = billboard_long.merge(
    billboard_songs, on=['year', 'artist', 'track', 'time'])
print(billboard_ratings.shape)

(24092, 8)


In [171]:
billboard_ratings = billboard_ratings.sort_values(by='id')

print(billboard_ratings.head())

       year artist                    track  time date.entered  week  rating  \
0      2000  2 Pac  Baby Don't Cry (Keep...  4:22   2000-02-26   wk1    87.0   
3487   2000  2 Pac  Baby Don't Cry (Keep...  4:22   2000-02-26  wk12     NaN   
14265  2000  2 Pac  Baby Don't Cry (Keep...  4:22   2000-02-26  wk46     NaN   
1902   2000  2 Pac  Baby Don't Cry (Keep...  4:22   2000-02-26   wk7    99.0   
951    2000  2 Pac  Baby Don't Cry (Keep...  4:22   2000-02-26   wk4    77.0   

       id  
0       0  
3487    0  
14265   0  
1902    0  
951     0  


Aparentemente os dados não estão ficando mais organizados da mesma forma que versões anteriores, então o jeito é se virar para formatar de outras formas

6.6 Unidades de observação em várias tabelas

A última porção da organização de dados relaciona-se à situação em que o mesmo tipo de dado está espelhado em vários conjuntos de dados diferentes.  
Isso pode de dar por diversos motivos, um deles é que separar os dados ocupa menos espaço em cada parte, isso é benefíco caso precise compartilhar esses dados pela internet.

Como merge e concatenação já foram discutidos esta seção terá foco em técnicas para carregar rapidamente várias fontes de dados e reuni-las

Usaremos o Unified New York City Taxi and Uber Data, um conjunto de dados com corridas de táxi e uber que é separado em 140 arquivos, porém usaremos apenas 5

Importante
-
O bagulho do livro não data funcionando, então peguei outros dados de táxi na internet

In [172]:
taxi1 = pd.read_parquet('../../data/green_tripdata_2015-01.parquet')
taxi2 = pd.read_parquet('../../data/green_tripdata_2015-02.parquet')
taxi3 = pd.read_parquet('../../data/green_tripdata_2015-03.parquet')
taxi4 = pd.read_parquet('../../data/green_tripdata_2015-04.parquet')
taxi5 = pd.read_parquet('../../data/green_tripdata_2015-05.parquet')

print(taxi1)
print(taxi1.columns)

print(taxi4)


         VendorID lpep_pickup_datetime lpep_dropoff_datetime  \
0               2  2015-01-01 00:31:10   2015-01-01 00:50:41   
1               2  2015-01-01 00:01:05   2015-01-01 00:03:30   
2               2  2015-01-01 00:09:01   2015-01-01 00:33:26   
3               2  2015-01-01 00:17:34   2015-01-01 00:27:07   
4               2  2015-01-01 00:32:38   2015-01-01 00:40:32   
...           ...                  ...                   ...   
1508488         2  2015-01-31 23:50:32   2015-02-01 00:05:43   
1508489         1  2015-01-31 23:20:01   2015-01-31 23:34:29   
1508490         1  2015-01-31 23:52:30   2015-02-01 00:18:53   
1508491         2  2015-01-31 23:47:51   2015-01-31 23:59:03   
1508492         2  2015-01-31 23:29:31   2015-01-31 23:29:37   

        store_and_fwd_flag  RatecodeID  PULocationID  DOLocationID  \
0                        N           1           255           234   
1                        N           1            75            74   
2                    

Removi as colunas que não irão ser usadas

In [173]:
taxi1 = taxi1[['VendorID','lpep_pickup_datetime','PULocationID']]
taxi2 = taxi2[['VendorID','lpep_pickup_datetime','PULocationID']]
taxi3 = taxi3[['VendorID','lpep_pickup_datetime','PULocationID']]
taxi4 = taxi4[['VendorID','lpep_pickup_datetime','PULocationID']]
taxi5 = taxi5[['VendorID','lpep_pickup_datetime','PULocationID']]


Renomeando as coluna para ficar mais facil de manipular

In [174]:
taxi1 = taxi1.rename(columns={'lpep_pickup_datetime': 'pickup_date', 'PULocationID': 'LocationID'})
taxi2 = taxi2.rename(columns={'lpep_pickup_datetime': 'pickup_date', 'PULocationID': 'LocationID'})
taxi3 = taxi3.rename(columns={'lpep_pickup_datetime': 'pickup_date', 'PULocationID': 'LocationID'})
taxi4 = taxi4.rename(columns={'lpep_pickup_datetime': 'pickup_date', 'PULocationID': 'LocationID'})
taxi5 = taxi5.rename(columns={'lpep_pickup_datetime': 'pickup_date', 'PULocationID': 'LocationID'})

In [175]:
print(taxi1.head(n=2), '\n')
print(taxi2.head(n=2), '\n')
print(taxi3.head(n=2), '\n')
print(taxi4.head(n=2), '\n')
print(taxi5.head(n=2), '\n')

   VendorID         pickup_date  LocationID
0         2 2015-01-01 00:31:10         255
1         2 2015-01-01 00:01:05          75 

   VendorID         pickup_date  LocationID
0         1 2015-02-01 00:47:01         145
1         2 2015-02-01 00:36:29         146 

   VendorID         pickup_date  LocationID
0         1 2015-03-01 00:32:19         145
1         1 2015-03-01 00:59:06         145 

   VendorID         pickup_date  LocationID
0         1 2015-04-01 00:26:00         145
1         2 2015-04-01 00:17:41         146 

   VendorID         pickup_date  LocationID
0         2 2015-05-01 00:24:18         146
1         2 2015-05-01 00:28:15         146 



In [176]:
print(taxi1.shape, '\n')
print(taxi2.shape, '\n')
print(taxi3.shape, '\n')
print(taxi4.shape, '\n')
print(taxi5.shape, '\n')

(1508493, 3) 

(1574830, 3) 

(1722574, 3) 

(1664394, 3) 

(1786848, 3) 



Os dados podem ser concatenados exatamento como fizemos no capítulo 4

In [177]:
taxi = pd.concat([taxi1, taxi2, taxi3, taxi4, taxi5])

print(taxi.shape)

(8257139, 3)


Entretando salvar manualmente cada dataframe se tornará tedioso quando os dados estiverem divididos em muitas partes.  
Como uma abordagem alternativa, podemos automatizar o processo usando laços e list comprehensions

6.6.1 Carregando vários dados usando um laço

6.6.2

Essas duas sessões foram apenas explicando como fazer concatenação dos dados a partir de um loop